In [93]:
# Homework 2 Part 2 (due 7/07/2024)

# Health-care assistance via probabilistic graphical modeling

### Objective
In this project, you will create a health-care assistance bot that can suggest diagnoses for a set of symptoms based on a probabilistic graphical model.

### Step 1: Review 
Review the code from the Bayesian networks exercise.

### Step 2: Acquire data
View this [research article](https://www.nature.com/articles/ncomms5212) and download its supplementary data sets 1, 2 and 3. These data sets include the occurrences of diseases, symptoms, and their co-occurrences in the scientific literature. (For the purpose of this exercise, we are going to assume that the frequency of co-occurrences of diseases and symptoms in scientific papers is proportional to the co-occurence frequencies of actual disease cases and symptoms.)

### Step 3: Create a Bayesian network
Using commands from the `pgmpy` library, create a Bayesian network in which the probability of exhibiting a symptom is conditional on the probability of having an associated disease. 

### Step 4: Initialize priors
Use the disease occurrence data to assign prior probabilities for diseases.

### Step 5: Calculate conditional probability tables
Use the co-occurrence data to define CPTs for each connected pair of disease and symptoms. (Hint: You may need to assign some occurrences of symptoms to an "idiopathic disease" to create valid CPTs.)

### Step 6:
Create a minimal interface in which your bot asks a users for a list of observed symptoms and then returns the name of the disease that is the most likely match to the symptoms. (Hint: Review the input/output commands that you have used in last week's homework.)

In [94]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np
import pandas as pd
import networkx as nx


# Load Data
sdc = pd.read_csv("s_d_coocurrence.txt", sep='\t',skiprows=1)
sym = pd.read_csv("symptom_occurence.txt", sep='\t',skiprows=1)
dis = pd.read_csv("disease_occurence.txt", sep='\t',skiprows=1)

In [95]:
# Code from Dr. Schwarze's announcement
def grabFromDF(df,value,iv_index=0,dv_index=1):
  
  sub_df = (df[df[df.columns[iv_index]]==value][df.columns[dv_index]])
  
  if len(sub_df):
    print(sub_df) # the error originates here I believe, it prints numbers for some reason
    return sub_df[1]
  else: return 0

# Code from Dr. Schwarze's announcement
def grabFromDF2(df,value1,value2,iv1_index=0,iv2_index=1,dv_index=2):
  var1 = df.columns[iv1_index]
  var2 = df.columns[iv2_index]
  var3 = df.columns[dv_index]

  sub_df = df[df[var1]==value1]
  subsub_df = sub_df[sub_df[var2]==value2]

  if len(subsub_df):
    return subsub_df[var3]
  else:
    return 0

# Code from Dr. Schwarze's announcement
def CPT2x2(disease_occ,symptom_occ,interaction_occ,total_disease_occ,total_symptom_occ,total_interaction_occ):
  p_disease = disease_occ/total_disease_occ
  p_joint = interaction_occ/total_interaction_occ

  pTT = (p_joint/p_disease if p_joint>0 else 0.0)
  pFT = 1 - pTT
  pTF = (symptom_occ-interaction_occ)/total_symptom_occ
  pFF = 1 - pTF

  return [pFF,pTF,pFT,pTT]

def driver():
  # Part 3: Define the structure of the Bayesian Network
  model = BayesianNetwork()
  # add disease nodes with occurrence over 500
  for row in range(dis.shape[0]):
    if dis.iloc[row,1] > 500:
      model.add_node(dis.iloc[row,0])

  # add symptom nodes and edges (with some failsafes)
  for row in range(sdc.shape[0]):
    if sdc.iloc[row,1] in model:
      model.add_node(sdc.iloc[row,0])
      if (sdc.iloc[row,1] != sdc.iloc[row,0]) and not(nx.has_path(model,sdc.iloc[row,0],sdc.iloc[row,1])):
        model.add_edge(sdc.iloc[row,1],sdc.iloc[row,0])

  # Part 4: Initialize Prior Probabilities (assuming user has a disease in the list)
  tot_sym_occ = sum(sym.iloc[:,1])
  tot_dis_occ = sum(dis.iloc[:,1]) # find total occurrences
  tot_int_occ = sum(sdc.iloc[:,2])

  # calculate prior probabilities
  for row in range(dis.shape[0]):
    if dis.iloc[row,0] in model:
      p = dis.iloc[row,1]/tot_dis_occ # probability has the disease
      cpd_d = TabularCPD(variable=dis.iloc[row,0],variable_card=2,values=[[1-p],[p]])
      model.add_cpds(cpd_d)

  # Part 5: Define CPTs for symptom nodes
  CPTs_symptoms = []
  for row in range(sym.shape[0]):
  # get all parent nodes
    if sym.iloc[row,0] in model:
      parents = list(model.predecessors(sym.iloc[row,0]))

      little_cpts = []
      for disease in parents:
        # occurrence of the selected disease
        dis_occ =grabFromDF(dis,disease)
        
        sym_occ = grabFromDF(sym,sym.iloc[row,0])
        
        int_occ = grabFromDF2(sdc,sym.iloc[row,0],disease)
        
        little_cpt = CPT2x2(dis_occ,sym_occ,int_occ,tot_dis_occ,tot_sym_occ,tot_int_occ)
        little_cpts += [little_cpt]
      rowT = []
      for bool_combo in itertools.product([0,1], repeat=len(parents)):
        cond_probs = [little_cpts[i][2+b] for i, b in enumerate(bool_combo)]
        rowT += [np.prod(cond_probs)]
      rowF = [1-val for val in rowT]

      cpt = TabularCPD(variable=sym.iloc[row,0],variable_card=2,values=[rowF,rowT],evidence=parents,evidence_card=[2 for _ in parents])
      CPTs_symptoms += [cpt]
  model.addcpds(CPTs_symptoms)

if __name__ == "__main__":
    driver()


44    28042
Name: 122226, dtype: int64


KeyError: 1

In [ ]:
# Create an inference object
inference = VariableElimination(model)

# Compute the probability of D given A=1 and B=0
prob_d_given_a1_b0 = inference.query(variables=['D'], evidence={'A': 1, 'B': 0})
print(prob_d_given_a1_b0)

# Compute the marginal probability of C
prob_c = inference.query(variables=['C'])
print(prob_c)